# Simularium Conversion Tutorial : ReaDDy Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.readdy import ReaddyConverter, ReaddyData
from simulariumio import UnitData, MetaData, DisplayData, DISPLAY_TYPE, ModelMetaData

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `ReaddyConverter` consumes spatiotemporal data from ReaDDy. 

The converter requires a `ReaddyData` object as a parameter ([see documentation](https://simularium.github.io/simulariumio/simulariumio.readdy.html#simulariumio.readdy.readdy_data.ReaddyData)).

If you'd like to specify PDB or OBJ files or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://simularium.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


In [2]:
box_size = 20.

example_data = ReaddyData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/simularium/simulariumio",
            source_code_license_url="https://github.com/simularium/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    timestep=0.1,
    path_to_readdy_h5="../simulariumio/tests/data/readdy/test.h5",
    display_data={
        "A": DisplayData(
            name="C",
            radius=2.0,
            display_type=DISPLAY_TYPE.OBJ,
            url="c.obj",
            color="#dfdacd",
        ),
        "B": DisplayData(
            name="B",
            radius=2.0,
            color="#0080ff",
        ),
    },
    ignore_types=[],  # list "A" or "B" here to leave that type out of the visualization
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

## Provide orientation data (optional)

If you want to calculate particle orientations for rendering PDB or mesh geometry for your ReaDDy topology particles, you can optionally provide orientation data.

To match the structure of a complex, e.g. a molecular complex from the [Protein Data Bank](https://www.rcsb.org/), measure the center of each coarse-grained particle you are defining from within the same structure file (e.g. use the biological assembly that includes all the structures your coarse-grained particles are based on). The distances and angles between the coarse-grained particles that you used to enforce potentials in ReaDDy should match the distances and angles between the particle positions you are using in this orientation data.

Here is an example of orientation data for a hypothetical D-C-A-B complex:

**Note:** Rotation calculation will fail for this example data because it does not include topologies.

In [3]:
"""
bond lengths:
  D - C : 1
  C - A : 1
  A - B : 2
  
z = 0

y
^         A ---- B
|         |
|    D -- C
|
|_ _ _ _ > x
"""
from simulariumio.readdy import OrientationData, NeighborData

example_data.zero_orientations = [
    OrientationData(
        type_name_substrings=["A"],
        neighbor_data=[
            NeighborData(
                type_name_substrings=["B"],
                relative_position=np.array([2, 0, 0]),
            ),
            NeighborData(
                type_name_substrings=["C"],
                relative_position=np.array([0, -1, 0]),
                neighbor_type_name_substrings=["D"],
                neighbor_relative_position=np.array([-1, -1, 0]),
            ),
        ],
    ),
    OrientationData(
        type_name_substrings=["C"],
        neighbor_data=[
            NeighborData(
                type_name_substrings=["A"],
                relative_position=np.array([0, 1, 0]),
            ),
            NeighborData(
                type_name_substrings=["D"],
                relative_position=np.array([-1, 0, 0]),
            ),
        ],
    ),
]

## Convert and save as .simularium file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [4]:
ReaddyConverter(example_data).save("example_readdy")

Reading ReaDDy Data -------------
Rotation calculation failed for B: couldn't find matching neighbor in ['A']
Rotation calculation failed for B: couldn't find matching neighbor in ['A']
Rotation calculation failed for B: couldn't find matching neighbor in ['A']
Rotation calculation failed for B: couldn't find matching neighbor in ['A']
Rotation calculation failed for B: couldn't find matching neighbor in ['A']
Rotation calculation failed for B: couldn't find matching neighbor in ['A']
Calculating rotations....
Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to example_readdy.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.